In [ ]:
# download presidio
!pip install presidio_analyzer presidio_anonymizer!python -m spacy download en_core_web_lg

Path to notebook: [https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/encrypt_decrypt.ipynb](https://www.github.com/microsoft/presidio/blob/main/docs/samples/python/encrypt_decrypt.ipynb)

# Encrypting and Decrypting identified entities

This sample shows how to use Presidio Anonymizer built-in functionality, to encrypt and decrypt identified entities.
The encryption is using AES cypher in CBC mode and requires a cryptographic key as an input for both the encryption and the decryption.


### Set up imports

In [2]:
from presidio_anonymizer import AnonymizerEngine, DeanonymizeEngine
from presidio_anonymizer.entities import RecognizerResult, OperatorResult, OperatorConfig
from presidio_anonymizer.operators import Decrypt

### Define a cryptographic key (for both encryption and decryption)

In [3]:
crypto_key = "WmZq4t7w!z%C&F)J"

### Presidio Anonymizer: Encrypt

In [4]:
engine = AnonymizerEngine()

# Invoke the anonymize function with the text,
# analyzer results (potentially coming from presidio-analyzer)
# and an 'encrypt' operator to get an encrypted anonymization output:
anonymize_result = engine.anonymize(
    text="My name is James Bond",
    analyzer_results=[
        RecognizerResult(entity_type="PERSON", start=11, end=21, score=0.8),
    ],
    operators={"PERSON": OperatorConfig("encrypt", {"key": crypto_key})},
)

anonymize_result

text: My name is M4lla0kBCzu6SwCONL6Y+ZqsPqhBp1Lhdc3t0FKnUwM=.
items:
[
    {'start': 11, 'end': 55, 'entity_type': 'PERSON', 'text': 'M4lla0kBCzu6SwCONL6Y+ZqsPqhBp1Lhdc3t0FKnUwM=', 'operator': 'encrypt'}
]

In [5]:
# Fetch the anonymized text from the result.
anonymized_text = anonymize_result.text

# Fetch the anonynized entities from the result.
anonymized_entities = anonymize_result.items

### Presidio Anonymizer: Decrypt

In [8]:
# Initialize the engine:
engine = DeanonymizeEngine()

# Invoke the deanonymize function with the text, anonymizer results
# and a 'decrypt' operator to get the original text as output.
deanonymized_result = engine.deanonymize(
    text=anonymized_text,
    entities=anonymized_entities,
    operators={"DEFAULT": OperatorConfig("decrypt", {"key": crypto_key})},
)

deanonymized_result

text: My name is James Bond.
items:
[
    {'start': 11, 'end': 21, 'entity_type': 'PERSON', 'text': 'James Bond', 'operator': 'decrypt'}
]

In [9]:
# Alternatively, call the Decrypt operator directly:

# Fetch the encrypted entitiy value from the previous stage
encrypted_entity_value = anonymize_result.items[0].text

# Restore the original entity value
Decrypt().operate(text=encrypted_entity_value, params={"key": crypto_key})

'James Bond'